In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, sys

from sklearn.model_selection import train_test_split
sys.path.append('../../Share/')
import Model

def random_downsample_Num(X, y=None, n_keep=None, random_state=42):
    np.random.seed(random_state)
    n_samples = X.shape[0]

    if n_keep is None or n_keep > n_samples:
        raise ValueError(f"n_keep must be between 1 and {n_samples}, got {n_keep}")

    idx_keep = np.random.choice(n_samples, size=n_keep, replace=False)
    X_down = X[idx_keep]

    if y is not None:
        y_down = y[idx_keep]
        return X_down, y_down
    else:
        return X_down


def baseline_run_specific_T_with_one_feature(X_lst, y_lst, trial, f=2):
    all_acc_per_session = []
    feature_set, num_channels = 14, 16

    for t in trial:
        X, y = X_lst[t].reshape(-1,num_channels,feature_set,1), y_lst[t]
        X_train = X[:,:,f,:]
        model = Model.Original_model_1DCNN(input_size=X_train.shape[1:], num_class=6)
        history, _ = Model.Train_model_without_test(model, X_train, y, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

    return model


def return_session_data(X_lst, y_lst, trial, f=2):
    all_acc_per_session = []
    feature_set, num_channels = 14, 16

    X, y = X_lst[trial].reshape(-1,num_channels,feature_set,1), y_lst[trial]
    X = X[:,:,f,:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    return X_train, X_test, y_train, y_test


def inter_session_run_one_feature(X_lst, y_lst, f=2):
    num_trials = len(X_lst)
    all_acc_K = []

    for t in [1, 4, 7, 9]:
        X_train = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t)]
        X_train = np.concatenate(X_train, axis=0)
        y_train = [y_lst[i] for i in range(t)]
        y_train = np.concatenate(y_train, axis=0)

        X_test = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t, len(X_lst))]
        X_test = np.concatenate(X_test, axis=0)

        y_test = [y_lst[i] for i in range(t, len(y_lst))]
        y_test = np.concatenate(y_test, axis=0)
        X_test, y_test = random_downsample_Num(X_test, y_test, n_keep=1000, random_state=42)
        X_train, X_test = X_train[:,:,f,:], X_test[:,:,f,:]
        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

        model = Model.Original_model_1DCNN(input_size=X_train.shape[1:], num_class=6)
        history, _ = Model.Train_model(model, X_train, y_train, X_test, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

        print(f"\tTrial {t+1}/{num_trials}/feat{f} | Val Acc: {np.max(history.history['val_accuracy'])}")
        all_acc_K.append(np.max(history.history['val_accuracy']))
    return all_acc_K


def generate_pseudo_label(model, x):
    y_pred = model.predict(x, verbose=0)
    return np.argmax(y_pred, axis=1)

def online_update(model, new_sample, pseudo_label, learning_rate=0.001):
    model.train_on_batch(new_sample, pseudo_label)
    return model

def evaluate_model(model, data, labels):
    return model.evaluate(data, labels, verbose=0)[1]

In [2]:
path = 'C:/Users/hml76/Desktop/Jupyter/Federated Prototype Learning/Dataset/Ours_cleaned/'
fname = os.listdir(path)

subjects = {
    "subject1": {"X": [], "Y": []},
    "subject2": {"X": [], "Y": []},
    "subject3": {"X": [], "Y": []},
}

for f in fname:
    for i in range(10):
        X = np.array(pd.read_csv(path + f'{f}/rep{i}_X.csv'))
        y = np.array(pd.read_csv(path + f'{f}/rep{i}_Y.csv'))

        # shuffle
        #indices = np.random.permutation(len(X))
        #X, y = X[indices], y[indices]

        # convert one-hot to integer labels
        y_int = np.argmax(y, axis=1)
        num_classes = np.max(y_int) + 1

        # example rule to assign data to subject dict
        if "subject1" in f:
            subjects["subject1"]["X"].append(X)
            subjects["subject1"]["Y"].append(y_int)
        elif "subject2" in f:
            subjects["subject2"]["X"].append(X)
            subjects["subject2"]["Y"].append(y_int)
        elif "subject3" in f:
            subjects["subject3"]["X"].append(X)
            subjects["subject3"]["Y"].append(y_int)

X_lst2, y_lst2 = subjects["subject2"]["X"], subjects["subject2"]["Y"]
X_lst3, y_lst3 = subjects["subject3"]["X"], subjects["subject3"]["Y"]

In [18]:
def online_learning_run(X_lst, y_lst, trials):
    model = baseline_run_specific_T_with_one_feature(X_lst, y_lst, trial=trials, f=2)

    Next_Session_Test_Acc, Same_Session_Test_Acc = [], []

    for trial in range(len(trials), len(X_lst)):
        X_train, X_test, y_train, y_test = return_session_data(X_lst, y_lst, trial, f=2)
        pseudo_labels = generate_pseudo_label(model, X_train)
        model = online_update(model, X_train, pseudo_labels)
        Same_Session_Test_Acc.append(evaluate_model(model, X_test, y_test))

        next_X, _, next_y, _ = return_session_data(X_lst, y_lst, trial, f=2)
        Next_Session_Test_Acc.append(evaluate_model(model, next_X, next_y))

    return Same_Session_Test_Acc, Next_Session_Test_Acc

In [25]:
a1,b1 = online_learning_run(X_lst2, y_lst2, trials=[0]) #1
a2,b2 = online_learning_run(X_lst2, y_lst2, trials=[0,1])  #2
a3,b3 = online_learning_run(X_lst2, y_lst2, trials=[0,1,2,3]) #4
a4,b4 = online_learning_run(X_lst2, y_lst2, trials=[0,1,2,3,4,5]) # 6
a5,b5 = online_learning_run(X_lst2, y_lst2, trials=[0,1,2,3,4,5,6,7]) # 8

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Traini

In [19]:
a1,b1 = online_learning_run(X_lst3, y_lst3, trials=[0]) #1
a2,b2 = online_learning_run(X_lst3, y_lst3, trials=[0,1])  #2
a3,b3 = online_learning_run(X_lst3, y_lst3, trials=[0,1,2,3]) #4
a4,b4 = online_learning_run(X_lst3, y_lst3, trials=[0,1,2,3,4,5]) # 6
a5,b5 = online_learning_run(X_lst3, y_lst3, trials=[0,1,2,3,4,5,6,7]) # 8

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Traini

In [33]:
a5,b5 = online_learning_run(X_lst3, y_lst3, trials=[0,1,2,3,4,5,6,7]) # 8

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)



In [ ]:
s2_same = [26.3, 29.0, 30.2, 30.7, 31.4]
s2_next = [26.6, 28.7, 29.2, 30.3, 31.5]

s3_same = [24.1, 32.4, 26.3, 42.5, 37.3]
s3_next = [23.6, 32.5, 27.0, 42.9, 37.2]

In [26]:
np.mean(a1)*100, np.mean(b1)*100

(np.float64(26.256696548726822), np.float64(26.613194909360672))

In [27]:
np.mean(a2)*100, np.mean(b2)*100

(np.float64(30.771242082118988), np.float64(30.93097973614931))

In [28]:
np.mean(a3)*100, np.mean(b3)*100

(np.float64(26.316284388303757), np.float64(25.74884494145711))

In [29]:
np.mean(a4)*100, np.mean(b4)*100

(np.float64(31.790482252836227), np.float64(31.10731914639473))

In [34]:
np.mean(a5)*100, np.mean(b5)*100

(np.float64(28.08738946914673), np.float64(28.67848575115204))